In [104]:
import pandas as pd
import pickle
import os
from datetime import date

%matplotlib inline

In [105]:
def track_sleep(weekly_sleep, existing=None, print_stats=False, save_dir=None):
    
    # explicitly order days of week
    days_ordered = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    
    if existing:
        # some checks to ensure existing sleep log is formatted correctly
        if not (isinstance(existing, pd.DataFrame) and all(existing.columns == days_ordered)
                and all([i.startswith('week') and i[-1].isdigit() for i in exising.index])):
            raise TypeError("You may only add to sleep logs output by this program")
        
        # continue week numbering from prior week...
        week_num = int(existing.index.tolist()[-1].split()[-1]) + 1
    
    else:
        # ... or start from scratch
        week_num = 1
    
    # convert input options to dataframe, with some checks
    if isinstance(weekly_sleep, list):
        if len(weekly_sleep) != 7:
            raise ValueError("Your list must have one entry for each day of the week")
        weekly_sleep = pd.DataFrame(weekly_sleep, columns=days_ordered, index=[week_num])
        
    elif isinstance(weekly_sleep, dict):
        weekly_sleep = pd.DataFrame(weekly_sleep, columns=days_ordered, index=[week_num])
        
    else:
        raise TypeError("You must pass either a dictionary or a Pandas DataFrame")
        
    if existing:
        new_log = existing.append(weekly_sleep)
    else:
        new_log = weekly_sleep
        
    if print_stats:
        message = 'You slept an average of {} hours this week'.format(new_log.tail(1).mean(axis=1))
        
        if existing:
            message += ', compared to {} hours last week'.format(new_log.iloc[-prev_wk, :].mean(axis=1))
            prev_wk = 3
            while prev_wk <= len(new_log):
                if prev_wk == len(new_log):
                    message += ', and '
                else:
                    message += ', '
                message += '{0} hours {1} weeks ago'.format(new_log.iloc[-prev_wk, :].mean(axis=1), prev_wk-1)
                
    if save_dir:
        with open(os.path.join(save_dir,'sleep_log_'+date.today().strftime("%m.%d.%y")+'.p'), 'wb') as f:
            pickle.dump(new_log, f)